In [ ]:
# !unzip /content/drive/MyDrive/MILDSum_test_468.zip -d /content/drive/MyDrive/MILDSum_test

In [ ]:
# !unzip /content/drive/MyDrive/MILDSum_train_2185.zip -d /content/drive/MyDrive/MILDSum_train

In [ ]:
# !unzip /content/drive/MyDrive/MILDSum_val_469.zip -d /content/drive/MyDrive/MILDSum_val

In [ ]:
pip install performer-pytorch

ERROR: unknown command "update"


In [ ]:
pip install pytorch-fast-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-fast-transformers: filename=pytorch_fast_transformers-0.4.0-cp311-cp311-linux_x86_64.whl size=21343742 sha256=073235c85272f9c1f64324f0195476ff0e67f9cbd1fc9d1c8d904af39a4a531d
  Stored in directory: /root/.cache/pip/wheels/64/0c/7d/39fe4c512d9a96aac3384d1506af78b0955e71e79163e459ec
Successfully built pytorch-fast-transformers


In [ ]:
import pickle

# Function to load pickle data
def load_pickle_data(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

# Load train data
train_eng_summary, train_hindi_summary = load_pickle_data("/content/drive/MyDrive/pick/train_data.pkl")

# Load test data
test_eng_summary, test_hindi_summary = load_pickle_data("/content/drive/MyDrive/pick/test_data.pkl")

# Load validation data
val_eng_summary, val_hindi_summary = load_pickle_data("/content/drive/MyDrive/pick/val_data.pkl")

# Print some sample data to verify
print("Train Data Samples:", train_eng_summary[:2], train_hindi_summary[:2])
print("Test Data Samples:", test_eng_summary[:2], test_hindi_summary[:2])
print("Validation Data Samples:", val_eng_summary[:2], val_hindi_summary[:2])

Train Data Samples: ['The Calcutta High Court has recently quashed criminal proceedings initiated against a lawyer for providing allegedly false and improper legal advice which was instrumental in sanction of a bank loan to a company which was later declared as Non Performing Asset (NPA) with an outstanding due of Rs. 2.57 crores. \nJustice Ananda Kumar Mukherjee observed that merely because the lawyer\'s opinion was not acceptable, criminal proceedings cannot be initiated against him especially in the absence of any tangible evidence that he was associated with other conspirators.\n"..it is beyond doubt that a lawyer owes an "unremitting loyalty" to the interests of the client and it is the lawyer\'s responsibility to act in a manner that would best advance the interest of the client. Merely because his opinion may not be acceptable, he cannot be mulcted with the criminal prosecution, particularly, in the absence of tangible evidence that he associated with other conspirators. At the 

In [ ]:
test_eng_summary[2]

'The Delhi High Court has observed that a momentary lapse on the part of the candidate must not be met with such a severe punitive action which would cause grave and irreparable prejudice and affect the candidate\'s future perversely.\nJustice Sanjeev Narula granted relief to a candidate namely Samriddhi Khandelwal by directing National Institute of Fashion Technology (NIFT) to allow her to join the counselling on the basis of her results of the online invigilated or remote proctored NIFT Entrance Exam, 2022. \nThe said Exam was divided into two parts   a written exam which was held on 06th February and a Situation Test which was to be held from 2nd April 2022 onwards. \nThe petitioner candidate had appeared for the online exam on 06th February 2022 from her residence, as per specifications mentioned in the Admit Card, complying with Rules and Regulations for Online Invigilated / Remote Proctored NIFT Entrance Exam for Admissions 2022. \nShe was then declared as "disqualified" in respe

# Custom Performer mt5

In [ ]:
import torch
from transformers import MT5Config, MT5Tokenizer, PreTrainedModel
from transformers.models.mt5.modeling_mt5 import MT5Stack, MT5ForConditionalGeneration
from performer_pytorch import SelfAttention
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import os

# Custom Performer-based Attention wrapper for integration
class PerformerSelfAttention(torch.nn.Module):
    def __init__(self, config, is_decoder=False):
        super().__init__()
        self.performer = SelfAttention(
            dim=config.d_model,
            heads=config.num_heads,
            nb_features=128,  # Reduced for memory efficiency
            causal=is_decoder
        )
        self.is_decoder = is_decoder
        self.dropout = torch.nn.Dropout(config.dropout_rate)
        self.output_attentions = config.output_attentions

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        position_bias=None,
        past_key_value=None,
        layer_head_mask=None,
        output_attentions=False,
        **kwargs
    ):
        # Apply Performer attention (without attention weights)
        hidden_states = self.performer(hidden_states)
        hidden_states = self.dropout(hidden_states)

        # Required outputs in MT5 format:
        # (hidden_states, present_key_value, position_bias, attention_weights, cross_attn_weights)
        return (
            hidden_states,     # hidden states after attention
            None,              # present_key_value (for cache)
            None,              # position_bias (used for relative positions in T5)
            None,              # attention_weights (Performer doesn't output them)
            None               # cross-attn weights (only relevant in decoder cross-attn)
        )

# Custom MT5 Encoder/Decoder block with Performer
class PerformerMT5Model(MT5ForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)
        self.config = config

        # Replace encoder self-attention with Performer
        for block in self.encoder.block:
            block.layer[0].SelfAttention = PerformerSelfAttention(config, is_decoder=False)

        # Replace decoder self-attention and cross-attention with Performer
        for block in self.decoder.block:
            block.layer[0].SelfAttention = PerformerSelfAttention(config, is_decoder=True)
            block.layer[1].EncDecAttention = PerformerSelfAttention(config, is_decoder=False)

        self.model = MT5Stack(config, self.shared)

    def forward(self, **kwargs):
        return super().forward(**kwargs)

In [ ]:
from transformers import MT5Tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "google/mt5-base"
tokenizer = MT5Tokenizer.from_pretrained(model_name, legacy=False)
config = MT5Config.from_pretrained(model_name)
tokenizer.model_max_length = 2048

# Update config for long input
config.max_length = 2048
config.attention_probs_dropout_prob = 0.1

# Initialize custom Performer MT5 model
model = PerformerMT5Model(config).to(device)
model.gradient_checkpointing_enable()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.


In [ ]:
def preprocess_data(eng_summary, hindi_summary, tokenizer):
    input_encodings = tokenizer(
        eng_summary,
        truncation=True,
        padding="longest",  # dynamically pads to the longest in the batch
        return_tensors="pt"
    )
    target_encodings = tokenizer(
        hindi_summary,
        truncation=True,
        padding="longest",
        return_tensors="pt"
    )

    return {
        'input_ids': input_encodings['input_ids'].squeeze(),
        'attention_mask': input_encodings['attention_mask'].squeeze(),
        'labels': target_encodings['input_ids'].squeeze()
    }

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, eng_summaries, hindi_summaries, tokenizer):
        self.eng_summaries = eng_summaries
        self.hindi_summaries = hindi_summaries
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.eng_summaries)

    def __getitem__(self, idx):
        return preprocess_data(
            self.eng_summaries[idx],
            self.hindi_summaries[idx],
            self.tokenizer
        )

In [ ]:
from transformers import DataCollatorForSeq2Seq

train_dataset = TranslationDataset(train_eng_summary, train_hindi_summary, tokenizer)
val_dataset = TranslationDataset(val_eng_summary, val_hindi_summary, tokenizer)
test_dataset = TranslationDataset(test_eng_summary, test_hindi_summary, tokenizer)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(val_dataset, batch_size=1, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=1, collate_fn=data_collator)

In [ ]:
optimizer = AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

In [ ]:
#from torch.amp import GradScaler

#gradient_accumulation_steps = 4  # Simulates batch_size=4 while using batch_size=2

def train_model(model, train_dataloader, val_dataloader, optimizer, scheduler, num_epochs=100, patience=4):
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    best_model_path = '/content/drive/MyDrive/pick/best_long_model.pt'
    scaler = GradScaler(enabled=True)

    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0

        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Mixed precision training
            #with autocast(device_type='cuda'):
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, return_dict=True)
            loss = outputs.loss

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()


        avg_train_loss = running_loss / len(train_dataloader)
        print(f"Training loss: {avg_train_loss}")

        val_loss = evaluate_model(model, val_dataloader)
        print(f"Validation loss: {val_loss}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"Saved best model at epoch {epoch+1}")
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

        scheduler.step(val_loss)

    print(f"Training complete. Best model saved at {best_model_path}")
    model.load_state_dict(torch.load(best_model_path))
    return model

In [ ]:
def evaluate_model(model, val_dataloader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Evaluating", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            total_loss += outputs.loss.item()

    avg_val_loss = total_loss / len(val_dataloader)
    return avg_val_loss

In [ ]:
# torch.cuda.empty_cache()

# # 3. Reset peak memory stats
# torch.cuda.reset_peak_memory_stats()

# print(torch.cuda.memory_allocated())  # This shows the amount of memory currently allocated
# print(torch.cuda.memory_reserved())   # This shows the amount of memory reserved by the CUDA memory allocator

8116401152
8378122240


In [ ]:
trained_model = train_model(model, train_dataloader, val_dataloader, optimizer, scheduler)

# Optionally, evaluate the best model on the test set
test_loss = evaluate_model(trained_model, test_dataloader)
print(f"Test Loss: {test_loss}")

RuntimeError: CUDA error: the provided PTX was compiled with an unsupported toolchain.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


# mt5-base

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import os

In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

# Device setup (prefer GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained M-T5 model and tokenizer
model_name = 'google/mt5-base'
model = MT5ForConditionalGeneration.from_pretrained(model_name).to(device)

# Use MT5Tokenizer explicitly
tokenizer = MT5Tokenizer.from_pretrained(model_name, legacy=False)

# Now you can use the tokenizer and model together

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.


In [ ]:
# Preprocessing function for tokenizing data
def preprocess_data(eng_summary, hindi_summary, tokenizer, max_input_length=512, max_output_length=128):
    input_encodings = tokenizer(eng_summary, truncation=True, padding='max_length', max_length=max_input_length, return_tensors="pt")
    target_encodings = tokenizer(hindi_summary, truncation=True, padding='max_length', max_length=max_output_length, return_tensors="pt")

    return {'input_ids': input_encodings['input_ids'].squeeze(),
            'attention_mask': input_encodings['attention_mask'].squeeze(),
            'labels': target_encodings['input_ids'].squeeze()}

In [ ]:
# Define custom dataset class
class TranslationDataset(Dataset):
    def __init__(self, eng_summaries, hindi_summaries, tokenizer, max_input_length=512, max_output_length=128):
        self.eng_summaries = eng_summaries
        self.hindi_summaries = hindi_summaries
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length

    def __len__(self):
        return len(self.eng_summaries)

    def __getitem__(self, idx):
        return preprocess_data(self.eng_summaries[idx], self.hindi_summaries[idx], self.tokenizer, self.max_input_length, self.max_output_length)

In [ ]:
# Prepare the datasets
train_dataset = TranslationDataset(train_eng_summary, train_hindi_summary, tokenizer)
val_dataset = TranslationDataset(val_eng_summary, val_hindi_summary, tokenizer)
test_dataset = TranslationDataset(test_eng_summary, test_hindi_summary, tokenizer)

# DataLoader setup
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1)
test_dataloader = DataLoader(test_dataset, batch_size=1)

In [ ]:
# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

In [ ]:
# Training loop with Early Stopping and ReduceLROnPlateau
def train_model(model, train_dataloader, val_dataloader, optimizer, scheduler, num_epochs=100, patience=4):
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    best_model_path = '/content/drive/MyDrive/pick/best_mt5_model.pt'

    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0

        # Training phase
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Backward pass
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_dataloader)
        print(f"Training loss: {avg_train_loss}")

        # Validation phase
        val_loss = evaluate_model(model, val_dataloader)
        print(f"Validation loss: {val_loss}")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"Saved best model at epoch {epoch+1}")
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        # Early stopping if no improvement
        if epochs_without_improvement >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

        # Adjust learning rate if necessary
        scheduler.step(val_loss)

    print(f"Training complete. Best model saved at {best_model_path}")

    # Load the best model
    model.load_state_dict(torch.load(best_model_path))
    return model

In [ ]:
# Evaluation loop
def evaluate_model(model, val_dataloader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Evaluating", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            total_loss += outputs.loss.item()

    avg_val_loss = total_loss / len(val_dataloader)
    return avg_val_loss

In [ ]:
# Start the training process
trained_model = train_model(model, train_dataloader, val_dataloader, optimizer, scheduler)

# Optionally, evaluate the best model on the test set
test_loss = evaluate_model(trained_model, test_dataloader)
print(f"Test Loss: {test_loss}")

Training loss: 3.094889259502053


Validation loss: 2.0382729042799044
Saved best model at epoch 1


Training loss: 1.6046608137047809


Validation loss: 1.3993469339150648
Saved best model at epoch 2


Training loss: 1.1302211129283468


...

Training loss: 0.2956916016865377


Validation loss: 1.2955959238844295


Training loss: 0.1459285473779219


Validation loss: 1.5707359995979528
Early stopping at epoch 8
Training complete. Best model saved at /content/drive/MyDrive/pick/best_mt5_model.pt


Test Loss: 1.0663208280108933


In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import torch

# Define model name
model_name = 'google/mt5-base'

# Load tokenizer
tokenizer = MT5Tokenizer.from_pretrained(model_name)

# Load the model architecture
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# Load the trained weights
model.load_state_dict(torch.load("/content/drive/MyDrive/pick/best_mt5_model.pt", map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu")))

# Move model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.


MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [ ]:
def split_text_into_segments(text, tokenizer, max_tokens=400):
    """
    Split text into segments of <= `max_tokens` (leaving room for special tokens).
    Uses sentence boundaries for better translation coherence.
    """
    sentences = text.split('. ')  # Simple split (improve with NLP libs like spaCy)
    segments = []
    current_segment = ""

    for sentence in sentences:
        sentence += ". "  # Re-add delimiter
        if len(tokenizer.encode(current_segment + sentence)) <= max_tokens:
            current_segment += sentence
        else:
            if current_segment:
                segments.append(current_segment.strip())
            current_segment = sentence

    if current_segment:
        segments.append(current_segment.strip())

    return segments

In [ ]:
def translate_long_text(model, tokenizer, english_text, device='cuda', max_input_tokens=512):
    # Split into manageable segments
    segments = split_text_into_segments(english_text, tokenizer, max_tokens=max_input_tokens)
    hindi_translations = []

    for segment in segments:
        inputs = tokenizer(
            segment,
            return_tensors="pt",
            truncation=True,
            max_length=max_input_tokens
        ).to(device)

        output_ids = model.generate(
            inputs.input_ids,
            max_length=150,
            num_beams=4,
            no_repeat_ngram_size=3,
            early_stopping=True
        )
        hindi_translation = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        hindi_translations.append(hindi_translation)

    # Combine translations (add space between segments)
    full_hindi_translation = " ".join(hindi_translations)
    return full_hindi_translation

In [ ]:
hindi_translation = translate_long_text(model, tokenizer, test_eng_summary[1], device)

import re

def clean_repetition(hindi_text):
    # Fix repeated phrases (e.g., "समाप्ति की समाप्ति...")
    cleaned_text = re.sub(r'(\b\w+\b)(?:\s+\1)+', r'\1', hindi_text)
    return cleaned_text

clean_translation = clean_repetition(hindi_translation)
print(clean_translation)

बॉम्बे हाईकोर्ट ने हाल ही में भारतीय दंड संहिता (आईपीसी) की धारा 302 के तहत बलात्कार का शिकार हुएक नाबालिग की 16 सप्ताह की गर्भपात की रोक लगाने की अनुमति दी। जस्टिस ए.एस. चंडूरकर और ज स्टिस यू.आर. जोशी-फाल्के की पीठ ने कहा कि सुप्रीम कोर्ट ने कहा है कि एक महिला के व्यक्तिगत स्वतंत्रता का एक अनिवार्य हिस्सा है, जैसा इलाहाबाद हाईकोर्ट ने कहा कि गर्भावस्था की समाप्ति की अनुमति दी जा सकती है, जब तक कि गर्भ की लंबाई दस सप्ताह नहीं हो जाती, तब तक गर्भवती महिला के जीवन या मानसिक स्वास्थ्य के लिएक गंभीर नुकसान हो। यदि गर्भ के लंबे समय के बीच प्रेगनेंसी की समय सीमा अधिक है, तो यह निर्धारित किया जा सकता है कि यह पूर्ण निष्पक्षता


In [ ]:
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')

def translate_long_text_mt5(input_text, tokenizer, model):
    sentences = sent_tokenize(input_text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        temp_chunk = current_chunk + " " + sentence if current_chunk else sentence
        if len(tokenizer.encode("translate English to Hindi: " + temp_chunk)) < 512:
            current_chunk = temp_chunk
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())

    # Translate chunks
    translations = []
    for chunk in chunks:
        prompt = "translate English to Hindi: " + chunk
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            padding="longest",
            max_length=512
        )
        outputs = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=256,  # safer than max_length
            num_beams=4,
            early_stopping=True,
            length_penalty=0.9,
            no_repeat_ngram_size=2
        )
        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        translations.append(translated_text.strip())

    return "\n".join(translations)


hindi_summary = translate_long_text_mt5(test_eng_summary[1], tokenizer, model)
print("English Summary:", test_eng_summary[1])
print("Hindi Summary:", hindi_summary)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


English Summary: The Bombay High Court recently allowed the termination of a 16 weeks' pregnancy of a minor who was a victim of sexual abuse and was also in custody at an Observation Home for a crime of murder under section 302 of the Indian Penal Code (IPC).
 A bench of Justices A.S. Chandurkar and Urmila Joshi-Phalke noted that the Apex Court has observed that reproductive choice is an insegragable part of a woman's personal liberty as envisaged under Article 21 of the Constitution of India.
"She cannot be forced to give birth to a child...She has a choice to give birth to the child or not," it observed.
The petitioner was a minor who had committed murder and was in custody at an Observation Home. It was discovered by the Investigating Officer that she was pregnant because of sexual abuse. A crime under Protection of Children from Sexual Offences Act, 2012 was registered.
 The petitioner pleaded that she is from an economically weak background, and has also undergone trauma due to se

In [ ]:
import re

def generate_summary(model, tokenizer, input_text, max_length=512):
    model.eval()  # Set model to evaluation mode
    inputs = tokenizer("Translate to hindi: "+input_text, return_tensors="pt", padding=True, truncation=True, max_length=max_length)

    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    print("Input token length:", len(inputs["input_ids"][0]))  # Debug token length
    # Generate output (Hindi summary)
    with torch.no_grad():
        output_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, num_beams=7, no_repeat_ngram_size=2, length_penalty = 0.5, max_length=max_length)
    print("Output token length:", len(output_ids[0]))  # Debug token length
    # Decode output tokens to Hindi text
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return summary


hindi_summary = generate_summary(model, tokenizer, test_eng_summary[2])

print("English Summary:", test_eng_summary[2])
print("Hindi Summary:", hindi_summary)

Input token length: 512
Output token length: 131
English Summary: The Bombay High Court recently allowed the termination of a 16 weeks' pregnancy of a minor who was a victim of sexual abuse and was also in custody at an Observation Home for a crime of murder under section 302 of the Indian Penal Code (IPC).
 A bench of Justices A.S. Chandurkar and Urmila Joshi-Phalke noted that the Apex Court has observed that reproductive choice is an insegragable part of a woman's personal liberty as envisaged under Article 21 of the Constitution of India.
"She cannot be forced to give birth to a child...She has a choice to give birth to the child or not," it observed.
The petitioner was a minor who had committed murder and was in custody at an Observation Home. It was discovered by the Investigating Officer that she was pregnant because of sexual abuse. A crime under Protection of Children from Sexual Offences Act, 2012 was registered.
 The petitioner pleaded that she is from an economically weak ba

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:

for i in sent_tokenize(test_eng_summary[2]):
    print(i)

    hindi_summary = generate_summary(model, tokenizer, i)
    print("hindi: ", hindi_summary)

    print("------------------------------------------------------------------------")


# tokenizer.tokenize()

The Delhi High Court has observed that a momentary lapse on the part of the candidate must not be met with such a severe punitive action which would cause grave and irreparable prejudice and affect the candidate's future perversely.
Input token length: 58
Output token length: 128
hindi:  दिल्ली हाईकोर्ट ने माना है कि किसी उम्मीदवार पर तत्काल देरी का विरोध करने की आवश्यकता नहीं होनी चाहिए, जिससे भाजपा की भविष्य में गंभीर और अप्रत्याशित हो। दिल्ली उच्च न्यायालय ने कहा कि पार्टी की ओर से होने वाला समय-सीमा के साथ ही यह कठोर दंडात्मक कार्रवाई को उससे कम से कम नजरअंदाज से नहीं किया जा सकता है, जो उम्मीदवारों
------------------------------------------------------------------------
Justice Sanjeev Narula granted relief to a candidate namely Samriddhi Khandelwal by directing National Institute of Fashion Technology (NIFT) to allow her to join the counselling on the basis of her results of the online invigilated or remote proctored NIFT Entrance Exam, 2022.
Input token length: 72
Output token l

removes repeated words

In [ ]:
hindi_summary

'बॉम्बे हाईकोर्ट ने हाल ही में एक नाबालिग की गर्भपात की रोक लगाने की अनुमति दी। नाबालिग की गर्भपात की रोक लगाने की अनुमति दी गई थी। जस्टिस एएस चंडूरकर और जस्टिस उर्मिला जोशी-फाल्के की पीठ ने कहा कि सुप्रीम कोर्ट ने माना है कि गर्भपात की स्वतंत्रता एक महिला के व्यक्तिगत स्वतंत्रता का एक अनिवार्य हिस्सा है। जस्टिस अश्विनी'

In [ ]:
import re

def remove_repeated_phrases(text):
    sentences = text.split("।")  # Splitting sentences for better phrase detection
    cleaned_sentences = []

    for i, sentence in enumerate(sentences):
        words = sentence.split()
        cleaned_words = []
        seen_phrases = set()

        j = 0
        while j < len(words):
            phrase = words[j]
            k = j + 1

            while k < len(words) and ' '.join(words[j:k+1]) in text:
                phrase = ' '.join(words[j:k+1])
                k += 1

            if phrase not in seen_phrases:
                cleaned_words.append(phrase)
                seen_phrases.add(phrase)

            j = k

        cleaned_sentence = ' '.join(cleaned_words)

        # Check if the sentence is mostly repeated from a previous one
        is_duplicate = any(
            len(set(cleaned_sentence.split()) & set(prev_sentence.split())) / len(set(cleaned_sentence.split())) > 0.7
            for prev_sentence in cleaned_sentences
        )

        if not is_duplicate:
            cleaned_sentences.append(cleaned_sentence)

    return '। '.join(cleaned_sentences).strip()

# Example usage:
text1 = "my name name name is xyz"
text2 = "there is a school a school a school bus"
text3 = "यह एक स्कूल स्कूल स्कूल बस है"
text4 = "बॉम्बे हाईकोर्ट ने हाल ही में एक नाबालिग की गर्भपात की रोक लगाने की अनुमति दी। नाबालिग की गर्भपात की रोक लगाने की अनुमति दी गई थी। जस्टिस एएस चंडूरकर और जस्टिस उर्मिला जोशी-फाल्के की पीठ ने कहा कि सुप्रीम कोर्ट ने माना है कि गर्भपात की स्वतंत्रता एक महिला के व्यक्तिगत स्वतंत्रता का एक अनिवार्य हिस्सा है। जस्टिस अश्विनी"


out1 = remove_repeated_phrases2(text1)
print( out1 )
print(remove_repeated_phrases(   out1  ))  # Output: "my name is xyz"
# print(remove_repeated_phrases(   remove_repeated_phrases2(text2)   ))  # Output: "there is a school bus"
# print(remove_repeated_phrases(   remove_repeated_phrases2(text3)    ))  # Output: "यह एक स्कूल बस है"
# print(remove_repeated_phrases(   remove_repeated_phrases2(text4   )) )


my name name name is xyz
my name name name is xyz


In [ ]:
import re

def remove_repeated_phrases2(text):
    def remove_repeated_words(sentence):
        words = sentence.split()
        cleaned_words = []
        i = 0
        while i < len(words):
            phrase = words[i]
            j = i + 1
            while j < len(words) and ' '.join(words[i:j+1]) in sentence:
                phrase = ' '.join(words[i:j+1])
                j += 1
            if not cleaned_words or phrase != cleaned_words[-1]:
                cleaned_words.append(phrase)
            i = j
        return ' '.join(cleaned_words)

    sentences = text.split("।")  # Splitting sentences for better phrase detection
    cleaned_sentences = []

    for i, sentence in enumerate(sentences):
        cleaned_sentence = remove_repeated_words(sentence.strip())

        # Check if the sentence is mostly repeated from a previous one
        is_duplicate = any(
            len(set(cleaned_sentence.split()) & set(prev_sentence.split())) / len(set(cleaned_sentence.split())) > 0.7
            for prev_sentence in cleaned_sentences
        )

        if not is_duplicate:
            cleaned_sentences.append(cleaned_sentence)

    return '। '.join(cleaned_sentences).strip()

# Example usage:
text1 = "my name name name is xyz"
text2 = "there is a school a school a school bus"
text3 = "यह एक स्कूल स्कूल स्कूल बस है"
text4 = "बॉम्बे हाईकोर्ट ने हाल ही में एक नाबालिग की गर्भपात की रोक लगाने की अनुमति दी। नाबालिग की गर्भपात की रोक लगाने की अनुमति दी गई थी। जस्टिस एएस चंडूरकर और जस्टिस उर्मिला जोशी-फाल्के की पीठ ने कहा कि सुप्रीम कोर्ट ने माना है कि गर्भपात की स्वतंत्रता एक महिला के व्यक्तिगत स्वतंत्रता का एक अनिवार्य हिस्सा है। जस्टिस अश्विनी"

print(remove_repeated_phrases2(text1))  # Output: "my name is xyz"
print(remove_repeated_phrases2(text2))  # Output: "there is a school bus"
print(remove_repeated_phrases2(text3))  # Output: "यह एक स्कूल बस है"
print(remove_repeated_phrases2(text4))


#   # Example usage:
# text1 = "my name name name is xyz"
# text2 = "there is a school a school a school bus"
# text3 = "यह एक स्कूल स्कूल स्कूल बस है"
# text4 = "बॉम्बे हाईकोर्ट ने हाल ही में एक नाबालिग की गर्भपात की रोक लगाने की अनुमति दी। नाबालिग की गर्भपात की रोक लगाने की अनुमति दी गई थी। जस्टिस एएस चंडूरकर और जस्टिस उर्मिला जोशी-फाल्के की पीठ ने कहा कि सुप्रीम कोर्ट ने माना है कि गर्भपात की स्वतंत्रता एक महिला के व्यक्तिगत स्वतंत्रता का एक अनिवार्य हिस्सा है। जस्टिस अश्विनी"

# remove_repeated_phrases2(text1)


my name name name is xyz
there is a school a school a school bus
यह एक स्कूल स्कूल स्कूल बस है
बॉम्बे हाईकोर्ट ने हाल ही में एक नाबालिग की गर्भपात की रोक लगाने की अनुमति दी। जस्टिस एएस चंडूरकर और जस्टिस उर्मिला जोशी-फाल्के की पीठ ने कहा कि सुप्रीम कोर्ट ने माना है कि गर्भपात की स्वतंत्रता एक महिला के व्यक्तिगत स्वतंत्रता का एक अनिवार्य हिस्सा है। जस्टिस अश्विनी


In [ ]:
def remove_repeated_phrases(text):
    words = text.split()
    cleaned_words = []

    i = 0
    while i < len(words):
        phrase = words[i]

        # Check for repeated sequences of words
        match = re.match(rf'({re.escape(phrase)}(?:\s+{re.escape(phrase)})+)', ' '.join(words[i:]))

        if match:
            cleaned_words.append(phrase)
            i += match.group(0).count(phrase)  # Skip repeated occurrences
        else:
            cleaned_words.append(phrase)
            i += 1

    return ' '.join(cleaned_words)

text1 = "my name name name is xyz"
text2 = "there is a school a school a school bus"
text3 = "यह एक स्कूल स्कूल स्कूल बस है"
text4 = "बॉम्बे हाईकोर्ट ने हाल ही में एक नाबालिग की गर्भपात की रोक लगाने की अनुमति दी। नाबालिग की गर्भपात की रोक लगाने की अनुमति दी गई थी। जस्टिस एएस चंडूरकर और जस्टिस उर्मिला जोशी-फाल्के की पीठ ने कहा कि सुप्रीम कोर्ट ने माना है कि गर्भपात की स्वतंत्रता एक महिला के व्यक्तिगत स्वतंत्रता का एक अनिवार्य हिस्सा है। जस्टिस अश्विनी"

print(remove_repeated_phrases(text1))  # Output: "my name is xyz"
print(remove_repeated_phrases(text2))  # Output: "there is a school bus"
print(remove_repeated_phrases(text3))  # Output: "यह एक स्कूल बस है"
print(remove_repeated_phrases(text4))

my name is xyz
there is a school a school a school bus
यह एक स्कूल बस है
बॉम्बे हाईकोर्ट ने हाल ही में एक नाबालिग की गर्भपात की रोक लगाने की अनुमति दी। नाबालिग की गर्भपात की रोक लगाने की अनुमति दी गई थी। जस्टिस एएस चंडूरकर और जस्टिस उर्मिला जोशी-फाल्के की पीठ ने कहा कि सुप्रीम कोर्ट ने माना है कि गर्भपात की स्वतंत्रता एक महिला के व्यक्तिगत स्वतंत्रता का एक अनिवार्य हिस्सा है। जस्टिस अश्विनी
